In [ ]:
import numpy as np
from numpy import asarray
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

from keras.models import load_model
from sklearn import preprocessing

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2
import random

In [ ]:
tf.__version__

In [ ]:
# avoid tensorflow to get all gpu memory
physical_devices = tf.config.list_physical_devices('GPU')
try:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
except:
    logger.info('error accessing gpu devices...')
    pass

In [ ]:
root_train = '../data/raw/vehicle/train/train/'

In [ ]:
data_train = []

In [ ]:
for category in sorted(os.listdir(root_train)):
    for file in sorted(os.listdir(os.path.join(root_train, category))):
        data_train.append((category, os.path.join(root_train, category, file)))

In [ ]:
train_df = pd.DataFrame(data_train, columns=['classes','file_path']).sample(frac=1.0)
train_df.head()

In [ ]:
cols=train_df['classes'].unique()
cols

In [ ]:
print("="*70)
print("Displaying 4 random image per vehicle category")
print("="*70)

# for every category in `cols`
for j in range(15):
    plt.figure(j)
    plt.figure(figsize=(20,20))
    
    # 4 images per every vehicle
    for i in range(4):
        id = "14{}".format(i+1)
        plt.subplot(int(id))
        random_file = random.choice(os.listdir(root_train + cols[j] + "/"))
        img = cv2.imread(root_train + cols[j] + "/" + random_file)
        plt.title(cols[j])
        plt.imshow(img)
plt.show()

In [ ]:
root_test = '../data/raw/vehicle/test/proof/'

In [ ]:
data_test = []
id_line = []

In [ ]:
count = 0
for file in sorted(os.listdir(os.path.join(root_test))):
    data_test.append((os.path.join(root_test, file)))
    id_line.append(count)
    count += 1

In [ ]:
test_df = pd.DataFrame(data_test, columns=['file_path'])
test_df.head()

In [ ]:
test_generator = ImageDataGenerator(rescale=1. / 255.)

In [ ]:
predict_generator = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col=None,
    batch_size=32,
    shuffle=False,
    class_mode=None,
    target_size= (224, 224))

### Testing with ResNet50 model

In [ ]:
name = 'complete_model_sequential.h5'

In [ ]:
network = load_model(f'../data/results/{name}')

In [ ]:
network.summary()

In [ ]:
steps = len(test_df) // 2

In [ ]:
proof = np.array(network.predict(predict_generator,
                                steps = steps,
                                verbose = 1,
                                workers = 10,
                                max_queue_size = 64))

In [ ]:
label_names = ['Ambulance','Bicycle','Boat','Bus','Car','Helicopter','Limousine',
               'Motorcycle','PickUp','Segway','Snowmobile','Tank','Taxi','Truck','Van']

#### testing one image

In [ ]:
test_df.iloc[0]

In [ ]:
simulation = proof[0]
print(simulation)

In [ ]:
classes = np.argmax(simulation, axis = 0)
print(classes)

In [ ]:
label_names[0]

#### testing the whole dataset

In [ ]:
class_names = []
for j in range(len(test_df)):
    b = np.argmax(proof[j], axis = 0)
    class_names.append(label_names[b])

In [ ]:
class_names

#### Plot the images with predicted label

In [ ]:
root_test = "../data/raw/vehicle/test/proof"

In [ ]:
import matplotlib.image as mpimg

In [ ]:
directory=sorted(os.listdir(root_test))
i=0
for each in directory:
    plt.figure(figsize=(8,8))
    img=mpimg.imread(f"../data/raw/vehicle/test/proof/{each}")
    imgplot = plt.imshow(img)
    plt.title(class_names[i])
    i+=1
plt.show()

### Testing with mobileNetV2 model

In [ ]:
name2 = 'complete_model_model.h5'

In [ ]:
network2 = load_model(f'../data/results/{name2}')

In [ ]:
network2.summary()

In [ ]:
proof2 = np.array(network2.predict(predict_generator,
                                   steps = steps,
                                   verbose = 1,
                                   workers = 10,
                                   max_queue_size = 64))

In [ ]:
class_names2 = []
for j in range(len(test_df)):
    b = np.argmax(proof2[j], axis = 0)
    class_names2.append(label_names[b])

In [ ]:
class_names2

In [ ]:
root_test = "../data/raw/vehicle/test/proof"

In [ ]:
import matplotlib.image as mpimg

In [ ]:
directory=sorted(os.listdir(root_test))
i=0
for each in directory:
    plt.figure(figsize=(8,8))
    img=mpimg.imread(f"../data/raw/vehicle/test/proof/{each}")
    plt.imshow(img)
    plt.title(class_names2[i])
    i+=1
plt.show()

In [ ]:
directory

### Plot the predictions

#### convert the image into array

In [ ]:
img_data = asarray(img)

In [ ]:
img_data = img_data / 255

In [ ]:
img_data=cv2.resize(img_data,(255,255))

#### convert the labels into array

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(class_names2)

In [ ]:
le.classes_

In [ ]:
label_data=le.transform(class_names2)
label_data

In [ ]:
# le.inverse_transform(label_data)

In [ ]:
def plot_image_pred(images: str,
                    labels: np.ndarray,
                    preds: np.ndarray,
                    indexes: list,
                    class_names: list = None, 
                    figsize: tuple = (8, 4)):
            
        for index in indexes:
        
            predicted_probas = preds[index]
            true_index = labels[index]
            predicted_index = np.argmax(predicted_probas)
            
            if class_names:
                true_class = class_names[true_index]
                predicted_class = class_names[predicted_index]

            fig, axes = plt.subplots(ncols=2, nrows=1, figsize=figsize)

            directory=sorted(os.listdir(root_test))
            axes[0].imshow(mpimg.imread(f"../data/raw/vehicle/test/proof/{directory[index]}"));
            axes[0].axis('off')
            axes[0].set_title(f"predicted class: {predicted_class if class_names else predicted_index}", 
                              color='blue' if true_index == predicted_index else 'red')

            axes[1].barh(class_names if class_names else [str(i) for i in range(len(predicted_probas))],
                         predicted_probas, 
                         color='black')
            axes[1].get_children()[predicted_index].set_color('red')
            axes[1].get_children()[true_index].set_color('blue')
            axes[1].set_xlim(0, 1)
            axes[1].set_title("class probabilities")
            axes[1].set_xlabel("probability")
            axes[1].set_ylabel("class name")


            plt.tight_layout()
            plt.show()

In [ ]:
plot_image_pred(root_test,
                label_data,
                proof2, 
                indexes=[20,21,22,23,24,25,26,27,28,19], 
                class_names=label_names)